In [1]:
pip install openpyxl requests pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\diego\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import requests
import pandas as pd
import json
import os
import shutil
from openpyxl import load_workbook

# ======================
# LOAD CREDENTIALS
# ======================
def load_blizzard_credentials(path):
    creds = {}
    print(f"🔐 Cargando credenciales desde: {path}")

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            key, value = line.split("=", 1)
            creds[key.strip()] = value.strip()

    if "CLIENT_ID" not in creds or "CLIENT_SECRET" not in creds:
        raise RuntimeError("❌ Credenciales inválidas")

    print("✅ Credenciales cargadas")
    return creds["CLIENT_ID"], creds["CLIENT_SECRET"]


# ======================
# DESKTOP DETECTION
# ======================
def find_desktop():
    home = os.path.expanduser("~")
    candidates = [
        os.path.join(home, "OneDrive", "Desktop"),
        os.path.join(home, "OneDrive", "Escritorio"),
    ]
    for path in candidates:
        if os.path.exists(path):
            print(f"📁 Escritorio detectado: {path}")
            return path
    raise RuntimeError("❌ No se encontró el Escritorio")

DESKTOP_PATH = find_desktop()
CREDENTIALS_FILE = os.path.join(DESKTOP_PATH, "blizzard_credentials.txt")
CLIENT_ID, CLIENT_SECRET = load_blizzard_credentials(CREDENTIALS_FILE)

# ======================
# CONFIG
# ======================
REGION = "us"
LOCALE = "en_US"

REALMS = {
    "Stormrage": 60,
    "Area 52": 3676,
    "Moon Guard": 3675,
    "Ragnaros": 1427,
    "Dalaran": 66,
    "Zul'jin":61,
    "Proudmoore":5
}

INPUT_XLSX = "Book.xlsx"
OUTPUT_XLSX = "BookUpdated.xlsx"
SHEET_NAME = "Hoja3"
CACHE_FILE = "item_id_cache.json"

# ======================
# AUTH
# ======================
def get_token():
    print("🔐 Obteniendo token Blizzard...")
    r = requests.post(
        f"https://{REGION}.battle.net/oauth/token",
        auth=(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type": "client_credentials"}
    )
    r.raise_for_status()
    print("✅ Token obtenido")
    return r.json()["access_token"]

# ======================
# CACHE
# ======================
def load_cache():
    return json.load(open(CACHE_FILE)) if os.path.exists(CACHE_FILE) else {}

def save_cache(cache):
    json.dump(cache, open(CACHE_FILE, "w"), indent=2)

# ======================
# EXCEL COPY
# ======================
def create_excel_copy(src, dst):
    shutil.copy(src, dst)
    print(f"📄 Copia creada: {dst}")

# ======================
# FIND TABLE
# ======================
def find_enchanting_table(path, sheet_name):
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None)

    for r in range(len(raw)):
        row = raw.iloc[r].astype(str).str.strip().str.lower().tolist()
        if "item" in row and "estimated sell price" in row:
            return r + 1, row.index("item") + 1, row.index("estimated sell price") + 1

    raise RuntimeError("❌ Tabla Enchanting no encontrada")

# ======================
# ITEM SEARCH
# ======================
def get_item_id(token, item_name, cache):
    if item_name in cache:
        return cache[item_name]

    r = requests.get(
        f"https://{REGION}.api.blizzard.com/data/wow/search/item",
        headers={"Authorization": f"Bearer {token}"},
        params={
            "namespace": f"static-{REGION}",
            "locale": LOCALE,
            "name.en_US": item_name,
            "_pageSize": 10
        }
    )
    r.raise_for_status()

    for res in r.json().get("results", []):
        name = res["data"]["name"].get(LOCALE)
        if name and name.lower() == item_name.lower():
            cache[item_name] = res["data"]["id"]
            return cache[item_name]

    return None

# ======================
# AUCTIONS
# ======================
def get_all_auctions(token, realms):
    all_auctions = {}
    for realm, realm_id in realms.items():
        print(f"📡 Descargando subastas de {realm}...")
        r = requests.get(
            f"https://{REGION}.api.blizzard.com/data/wow/connected-realm/{realm_id}/auctions",
            headers={"Authorization": f"Bearer {token}"},
            params={"namespace": f"dynamic-{REGION}", "locale": LOCALE}
        )
        r.raise_for_status()
        all_auctions[realm] = r.json()["auctions"]
    return all_auctions

def get_price_per_unit(a):
    if a.get("unit_price"):
        return a["unit_price"]
    if a.get("buyout"):
        return a["buyout"] / a.get("quantity", 1)
    return None

def min_buyout(auctions, item_id):
    prices = [
        get_price_per_unit(a)
        for a in auctions
        if a.get("item", {}).get("id") == item_id and get_price_per_unit(a)
    ]
    return min(prices) if prices else None

def min_buyout_by_realm(all_auctions, item_id):
    prices = {}
    for realm, auctions in all_auctions.items():
        p = min_buyout(auctions, item_id)
        if p:
            prices[realm] = p
    return prices

def best_realm_to_sell(prices):
    if not prices:
        return None, None
    best = max(prices, key=prices.get)
    return best, prices[best]

# ======================
# MAIN
# ======================
# ======================
# MAIN
# ======================
# ======================
# MAIN
# ======================
def main():
    create_excel_copy(INPUT_XLSX, OUTPUT_XLSX)

    token = get_token()
    all_auctions = get_all_auctions(token, REALMS)
    cache = load_cache()

    header_row, col_item, col_price = find_enchanting_table(OUTPUT_XLSX, SHEET_NAME)

    wb = load_workbook(OUTPUT_XLSX)
    ws = wb[SHEET_NAME]

    # Columna nueva: Best Realm (si no existe)
    col_best_realm = col_price + 1
    ws.cell(row=header_row, column=col_best_realm, value="Best Realm")

    r = header_row + 1
    updated = 0

    print("\n🚀 Analizando precios multi-reino...\n")

    while True:
        item_cell = ws.cell(row=r, column=col_item)
        price_cell = ws.cell(row=r, column=col_price)
        best_realm_cell = ws.cell(row=r, column=col_best_realm)

        if not item_cell.value:
            break

        item_name = str(item_cell.value).strip()
        print(f"➡ {item_name}")

        item_id = get_item_id(token, item_name, cache)
        if not item_id:
            print("   ❌ Item no encontrado")
            r += 1
            continue

        prices = min_buyout_by_realm(all_auctions, item_id)
        best_realm, best_price = best_realm_to_sell(prices)

        if best_price:
            gold = round(best_price / 10000, 2)
            price_cell.value = gold  # Actualizamos el Estimated Sell Price
            best_realm_cell.value = best_realm  # Actualizamos Best Realm
            updated += 1

            print("   📊 Precios:")
            for realm, p in prices.items():
                print(f"      {realm}: {round(p / 10000, 2)}g")
            print(f"   🏆 Mejor vender en {best_realm} → {gold}g")
        else:
            print("   ⚠ Sin subastas")
            best_realm_cell.value = "N/A"

        r += 1

    save_cache(cache)
    wb.save(OUTPUT_XLSX)

    print("\n============================")
    print(f"✔ {updated} items actualizados")
    print(f"✔ Archivo generado: {OUTPUT_XLSX}")
    print("============================")



main()


📁 Escritorio detectado: C:\Users\diego\OneDrive\Desktop
🔐 Cargando credenciales desde: C:\Users\diego\OneDrive\Desktop\blizzard_credentials.txt
✅ Credenciales cargadas
📄 Copia creada: BookUpdated.xlsx
🔐 Obteniendo token Blizzard...
✅ Token obtenido
📡 Descargando subastas de Stormrage...
📡 Descargando subastas de Area 52...
📡 Descargando subastas de Moon Guard...
📡 Descargando subastas de Ragnaros...
📡 Descargando subastas de Zul'jin...
📡 Descargando subastas de Proudmoore...

🚀 Analizando precios multi-reino...

➡ Replica Awakening Machine Stasis Pod
   📊 Precios:
      Stormrage: 8998.0g
      Area 52: 6000.94g
      Moon Guard: 10230.93g
      Ragnaros: 7800.0g
      Zul'jin: 7500.42g
      Proudmoore: 15800.99g
   🏆 Mejor vender en Proudmoore → 15800.99g
➡ Dornogal Hanging Sconce
   📊 Precios:
      Stormrage: 3600.82g
      Area 52: 2500.5g
      Moon Guard: 3090.0g
      Ragnaros: 2499.0g
      Zul'jin: 2925.0g
      Proudmoore: 3900.98g
   🏆 Mejor vender en Proudmoore → 3900.98g
